### Portfolio-Split

In [40]:
import pandas as pd
df = pd.read_excel(r"C:\Programming\python\PyProtolinc\examples\04_two_state_disability\portfolio\portfolio_small3.xlsx")
df

,DATE_PORTFOLIO,ID,DATE_OF_BIRTH,DATE_START_OF_COVER,SUM_INSURED,CURRENT_STATUS,SEX,PRODUCT,PRODUCT_PARAMETERS,SMOKERSTATUS,RESERVING_RATE,DATE_OF_DISABLEMENT
0,2021-12-31,1,1976-04-23,2022-01-01,1000,ACTIVE,m,TwoStateDisability,NaN,U,0,NaT
1,2021-12-31,2,1969-03-03,2022-01-01,100,ACTIVE,f,TwoStateDisability,NaN,U,0,NaT
2,2021-12-31,3,1970-09-13,2022-01-01,5000,DIS1,f,TwoStateDisability,NaN,U,0,2021-12-31


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   DATE_PORTFOLIO       3 non-null      datetime64[ns]
 1   ID                   3 non-null      int64         
 2   DATE_OF_BIRTH        3 non-null      datetime64[ns]
 3   DATE_START_OF_COVER  3 non-null      datetime64[ns]
 4   SUM_INSURED          3 non-null      int64         
 5   CURRENT_STATUS       3 non-null      object        
 6   SEX                  3 non-null      object        
 7   PRODUCT              3 non-null      object        
 8   PRODUCT_PARAMETERS   0 non-null      float64       
 9   SMOKERSTATUS         3 non-null      object        
 10  RESERVING_RATE       3 non-null      int64         
 11  DATE_OF_DISABLEMENT  0 non-null      float64       
dtypes: datetime64[ns](3), float64(2), int64(3), object(4)
memory usage: 416.0+ bytes


In [49]:
# df.DATE_OF_BIRTH.info()
import numpy as np
x = df.DATE_OF_BIRTH.dt

df.DATE_OF_BIRTH.dtype # == np.datetime64
#x.year
#x.info()

the_date = pd.DatetimeIndex(df.DATE_PORTFOLIO.astype("datetime64[ns]").values)

df.DATE_OF_BIRTH.astype("datetime64[ns]").values
date_col = pd.DatetimeIndex(df.DATE_OF_DISABLEMENT.astype("datetime64[ns]").values)

date_col, the_date

(DatetimeIndex(['NaT', 'NaT', '2021-12-31'], dtype='datetime64[ns]', freq=None),
 DatetimeIndex(['2021-12-31', '2021-12-31', '2021-12-31'], dtype='datetime64[ns]', freq=None))

In [54]:
age_in_months0 = \
        the_date.year * 12 + the_date.month \
        - date_col.year * 12 - date_col.month \
        - 1 \
        + (the_date.day >= date_col.day).astype(int)

age_in_months0.fillna(-1)

Float64Index([-1.0, -1.0, 0.0], dtype='float64')

In [50]:
# load a portfolio
from pyprotolinc.models.model_disability_multistate import MultiStateDisabilityStates
from pyprotolinc.portfolio import Portfolio
portfolio_path = r"D:\programming\PyMultiState\pyprotolinc\data\portfolio\portfolio_med.xlsx"

# load the portfolio
portfolio = Portfolio(portfolio_path, MultiStateDisabilityStates)
portfolio

ModuleNotFoundError: No module named 'pyprotolinc.models.model_disability_multistate'

In [2]:
# the initial ages in (completed) months
portfolio.initial_ages

array([548, 633, 615, ..., 633, 615, 548], dtype=int64)

In [3]:
# IDEA: the assumptions are determined on a yearly basis so that they remain constant for 12 months
# for each insured but at which months they switch depends on the insured. More specifically, it depends on the age
# definition. Based on what we have so far that would depend on the age in months and months when assumptions 
# switch is the same for all records which differ in age my full years.

In [4]:
ages_groups = portfolio.initial_ages % 12
ages_groups

array([8, 9, 3, ..., 9, 3, 8], dtype=int64)

In [5]:
# illustration
import pandas as pd
df = pd.DataFrame({"ages": portfolio.initial_ages, "age_groups": ages_groups})
print(df["age_groups"].unique())
df.head()

[8 9 3]


,ages,age_groups
0,548,8
1,633,9
2,615,3
3,548,8
4,633,9


In [6]:
import numpy as np
month_groups, group_inverses = np.unique(ages_groups, return_inverse=True)
month_groups, group_inverses

(array([3, 8, 9], dtype=int64), array([1, 2, 0, ..., 2, 0, 1], dtype=int64))

In [7]:
# an index for the parent
portfolio_index = np.arange(len(portfolio.initial_ages))
portfolio_index

array([   0,    1,    2, ..., 9997, 9998, 9999])

In [8]:
sub_portfolio_indexes = [portfolio_index[ages_groups == k] for k in month_groups]

In [9]:
sub_portfolio_indexes

[array([   2,    5,    8, ..., 9992, 9995, 9998]),
 array([   0,    3,    6, ..., 9993, 9996, 9999]),
 array([   1,    4,    7, ..., 9991, 9994, 9997])]

In [10]:
# now use the functionality
subportfolios = portfolio.split_by_month_in_year()

In [11]:
subportfolios

In [12]:
[len(sp) for sp in subportfolios]

[3333, 3334, 3333]

In [16]:
[sp.common_month for sp in subportfolios], portfolio.common_month is None

([3, 8, 9], True)

True

In [18]:
for k, l in enumerate(('A', 'B')):
    print(k, l)

0 A
1 B
